In [34]:
import pandas as pd
df = pd.read_csv("DataScienceQA.csv", encoding='unicode_escape')

In [35]:
question =  df['Question'].tolist()

In [36]:
answer =  df['Answer'].tolist()

In [37]:
import nltk
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\lenovo\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\lenovo\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\lenovo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [38]:
from nltk.stem import WordNetLemmatizer, PorterStemmer
from nltk.corpus import stopwords
import re

def preprocess(text):
    lemmatizer = WordNetLemmatizer()
    stemmer = PorterStemmer()
    text = re.sub(r'[^\w\s]', '', text)  # Remove non-alphanumeric characters
    tokens = nltk.word_tokenize(text.lower())
    tokens = [token for token in tokens if token not in stopwords.words('english')]
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in tokens]
    stemmed_tokens = [stemmer.stem(token) for token in lemmatized_tokens]
    return ' '.join(stemmed_tokens)

In [39]:
def preprocess_with_stopwords(text):
    lemmatizer = WordNetLemmatizer()
    stemmer = PorterStemmer()
    text = re.sub(r'[^\w\s]', '', text)  # Remove non-alphanumeric characters
    tokens = nltk.word_tokenize(text.lower())
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in tokens]
    stemmed_tokens = [stemmer.stem(token) for token in lemmatized_tokens]
    return ' '.join(stemmed_tokens)

In [40]:
vectorizer = TfidfVectorizer(tokenizer=nltk.word_tokenize)
X = vectorizer.fit_transform([preprocess(q) for q in question])

c:\Users\lenovo\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [41]:
def get_response(text):
    # Verify the preprocessed text
    processed_text = preprocess_with_stopwords(text)
    print("Processed text:", processed_text)

    # Vectorize the text and check for valid transformation
    vectorized_text = vectorizer.transform([processed_text])

    # Check cosine similarity and ensure consistent feature space
    similarities = cosine_similarity(vectorized_text, X)

    max_similarity = np.max(similarities)
    print("Max similarity:", max_similarity)

    if max_similarity > 0.6:
        high_similarity_questions = [q for q, s in zip(question, similarities[0]) if s > 0.6]
        print("High similarity questions:", high_similarity_questions)

        # Ensure index retrieval is correct
        target_answers = []
        for q in high_similarity_questions:
            q_index = question.index(q)
            target_answers.append(answer[q_index])

        Z = vectorizer.transform([preprocess_with_stopwords(q) for q in high_similarity_questions])
        processed_text_with_stopwords = preprocess_with_stopwords(text)
        vectorized_text_with_stopwords = vectorizer.transform([processed_text_with_stopwords])

        # Verify final similarity calculation
        final_similarities = cosine_similarity(vectorized_text_with_stopwords, Z)
        closest = np.argmax(final_similarities)

        return target_answers[closest]

    else:
        return "I can't answer this question."
